In [3]:

%load_ext autoreload
%autoreload 2


In [8]:
import torch
import torchvision
import torchvision.transforms as transforms
from utils import *
import torch.optim as optim
from model.baseline_model import *
from trainer import *
from torch.utils.tensorboard import SummaryWriter
import robustbench
#from robustbench.data import load_cifar10
from robustbench.utils import load_model, clean_accuracy

import warnings
warnings.filterwarnings('ignore')

In [5]:
train_dataset, val_dataset, train_loader, val_loader = CIFAR10()

0it [00:00, ?it/s]

Extracting ./datasets/CIFAR-10/cifar-10-python.tar.gz to ./datasets/CIFAR-10
Files already downloaded and verified


In [6]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
device = "cuda"



# Train shallow CNN model 

In [9]:
from torch.utils.tensorboard import SummaryWriter
net_ori = BaseNet().to(device)
train_criterion = nn.CrossEntropyLoss()
val_criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net_ori.parameters(), lr=0.001, momentum=0.9)
PATH = './cifar_base.pth'
device = "cuda"
writer = SummaryWriter("NormalBase")

In [10]:
train(net_ori, train_loader, train_criterion,val_loader, val_criterion, 
              optimizer, 100, writer, 
                 PATH, False, None, device="cuda",
                  n_steps_show=1000)

Epoch [0/100]	
Epoch [1/100]	 Training Loss: 2.3029514401770004	 lr: 0.001
Epoch [1/100]	 Validation Loss: 2.3015229158763644	 lr: 0.001
Epoch [1/100]	
Epoch [2/100]	 Training Loss: 2.3000576453440633	 lr: 0.001
Epoch [2/100]	 Validation Loss: 2.297634154935426	 lr: 0.001
Epoch [2/100]	
Epoch [3/100]	 Training Loss: 2.2934116225718233	 lr: 0.001
Epoch [3/100]	 Validation Loss: 2.2857805233967454	 lr: 0.001
Epoch [3/100]	
Epoch [4/100]	 Training Loss: 2.270483707223097	 lr: 0.001
Epoch [4/100]	 Validation Loss: 2.244015521641019	 lr: 0.001
Epoch [4/100]	
Epoch [5/100]	 Training Loss: 2.1838065199839796	 lr: 0.001
Epoch [5/100]	 Validation Loss: 2.0896584202971638	 lr: 0.001
Epoch [5/100]	
Epoch [6/100]	 Training Loss: 2.0403649858806445	 lr: 0.001
Epoch [6/100]	 Validation Loss: 2.0033185255678396	 lr: 0.001
Epoch [6/100]	
Epoch [7/100]	 Training Loss: 1.9974482367410684	 lr: 0.001
Epoch [7/100]	 Validation Loss: 1.9824071051199226	 lr: 0.001
Epoch [7/100]	
Epoch [8/100]	 Training Loss:

In [11]:
#test clean accuray
test(val_loader, net_ori)


Accuracy of the network on the 10000 test images: 60 %


In [12]:
#net = BaseNet().to(device)
#test attacks 
pgd_attack = generate_adv(net_ori, "pgd")
fgsm_attack = generate_adv(net_ori, "fgsm")
nifgsm_attack = generate_adv(net_ori, "nifgsm")
vmifgsm_attack = generate_adv(net_ori, "vmifgsm")




In [13]:
images, labels = load_dataset(10000, val_loader)

In [14]:
adv_images_pgd_adv = pgd_attack(images, labels)
acc = clean_accuracy(net_ori, adv_images_pgd_adv, labels)
print('[Model loaded]')
print('Acc: %2.2f %%'%(acc*100))
print("adv model")

[Model loaded]
Acc: 1.02 %
adv model


In [15]:
adv_images_fgsm_adv = fgsm_attack(images, labels)
acc = clean_accuracy(net_ori, adv_images_fgsm_adv, labels)
print('[Model loaded]')
print('Acc: %2.2f %%'%(acc*100))
print("adv model")

[Model loaded]
Acc: 3.65 %
adv model


In [16]:
#nifgsm_attack = generate_adv(net, "nifgsm")

adv_images_nifgsm_adv = nifgsm_attack(images, labels)
acc = clean_accuracy(net_ori, adv_images_nifgsm_adv, labels)
print('[Model loaded]')
print('Acc: %2.2f %%'%(acc*100))
print("adv model")

[Model loaded]
Acc: 1.88 %
adv model


In [17]:
#vmifgsm_attack = generate_adv(net, "vmifgsm")
adv_images_vmifgsm_adv =vmifgsm_attack(images, labels)
acc = clean_accuracy(net_ori, adv_images_vmifgsm_adv, labels)
print('[Model loaded]')
print('Acc: %2.2f %%'%(acc*100))
print("adv model")

[Model loaded]
Acc: 1.09 %
adv model


# Adversarial Training on shallow CNN model by PGD

In [18]:
from torch.utils.tensorboard import SummaryWriter
import copy
net_adv = copy.deepcopy(net_ori) #am I doing correct? 
for param in net_adv.parameters():
    print(param)

Parameter containing:
tensor([[[[ 8.5394e-02, -1.1512e-01, -1.0861e-01, -1.9637e-02, -1.8678e-02],
          [ 3.7605e-02, -6.2260e-02, -5.1802e-02,  5.4722e-02, -5.2045e-02],
          [-3.0720e-02,  1.0825e-01, -7.0859e-02, -6.5885e-02,  4.8625e-02],
          [ 8.8407e-02, -3.9397e-02, -4.9881e-02,  1.5771e-02, -4.9440e-02],
          [-1.0586e-01, -6.3314e-02,  5.0533e-02,  1.1038e-01, -6.0548e-02]],

         [[-9.8391e-02, -3.9039e-02, -3.9520e-02, -7.5328e-02, -6.3877e-02],
          [-6.8654e-02,  3.4659e-02, -8.7734e-02, -7.3791e-02,  5.3084e-02],
          [-8.5928e-02, -1.0200e-01, -9.4525e-02,  9.3352e-02,  9.8604e-03],
          [ 3.7303e-03, -9.6609e-02, -4.9321e-02, -1.0189e-01, -7.0317e-02],
          [-8.2237e-02, -5.6334e-02,  3.2203e-02,  3.9847e-02,  1.0035e-01]],

         [[ 2.1195e-02, -1.9120e-03,  5.2093e-02, -6.7285e-02,  1.0167e-01],
          [-6.7320e-02, -4.1070e-02,  7.0719e-02,  9.2750e-02, -4.3728e-02],
          [-6.5170e-02, -1.5343e-02,  6.9010e-02, 

In [19]:
train_criterion = nn.CrossEntropyLoss()
val_criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net_adv.parameters(), lr=0.001, momentum=0.9)
PATH = './cifar_base_adv_pgd.pth'
device = "cuda"
writer = SummaryWriter("Adv_Base_PGD")
pgd_attack = generate_adv(net_adv, "pgd")

In [20]:
train(net_adv, train_loader, train_criterion,val_loader, val_criterion, 
              optimizer, 100, writer, 
                 PATH, True, pgd_attack, device="cuda",
                  n_steps_show=1000)

Epoch [0/100]	
Epoch [1/100]	 Training Loss: 2.606929569293166	 lr: 0.001
Epoch [1/100]	 Validation Loss: 1.7744153255148778	 lr: 0.001
Epoch [1/100]	
Epoch [2/100]	 Training Loss: 2.206723759241421	 lr: 0.001
Epoch [2/100]	 Validation Loss: 1.7443217552160915	 lr: 0.001
Epoch [2/100]	
Epoch [3/100]	 Training Loss: 2.1409123800599668	 lr: 0.001
Epoch [3/100]	 Validation Loss: 1.7191773788838447	 lr: 0.001
Epoch [3/100]	
Epoch [4/100]	 Training Loss: 2.1064195931720002	 lr: 0.001
Epoch [4/100]	 Validation Loss: 1.7154925005345405	 lr: 0.001
Epoch [4/100]	
Epoch [5/100]	 Training Loss: 2.0841019842630764	 lr: 0.001
Epoch [5/100]	 Validation Loss: 1.6828070652635791	 lr: 0.001
Epoch [5/100]	
Epoch [6/100]	 Training Loss: 2.064460202556132	 lr: 0.001
Epoch [6/100]	 Validation Loss: 1.6758251235454897	 lr: 0.001
Epoch [6/100]	
Epoch [7/100]	 Training Loss: 2.051415484579628	 lr: 0.001
Epoch [7/100]	 Validation Loss: 1.6651591288892529	 lr: 0.001
Epoch [7/100]	
Epoch [8/100]	 Training Loss: 

In [21]:
#test clean accuray
test(val_loader, net_adv)

Accuracy of the network on the 10000 test images: 48 %


In [22]:
acc = clean_accuracy(net_adv, adv_images_fgsm_adv, labels)
print('[Model loaded]')
print('Acc: %2.2f %%'%(acc*100))
print("adv model")

[Model loaded]
Acc: 43.13 %
adv model


In [23]:
acc = clean_accuracy(net_adv, adv_images_vmifgsm_adv, labels)
print('[Model loaded]')
print('Acc: %2.2f %%'%(acc*100))
print("adv model")

[Model loaded]
Acc: 42.59 %
adv model


In [24]:
acc = clean_accuracy(net_adv, adv_images_nifgsm_adv, labels)
print('[Model loaded]')
print('Acc: %2.2f %%'%(acc*100))
print("adv model")

[Model loaded]
Acc: 42.91 %
adv model


In [25]:
acc = clean_accuracy(net_adv, adv_images_pgd_adv, labels)
print('[Model loaded]')
print('Acc: %2.2f %%'%(acc*100))
print("adv model")

[Model loaded]
Acc: 43.04 %
adv model


# Adversarial Training by nifgsm

In [30]:
from torch.utils.tensorboard import SummaryWriter
import copy
net_adv_ni = copy.deepcopy(net_ori) #am I doing correct? 

train_criterion = nn.CrossEntropyLoss()
val_criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net_adv_ni.parameters(), lr=0.001, momentum=0.9)
PATH = './cifar_base_adv_nifgsm.pth'
device = "cuda"
writer = SummaryWriter("Adv_Base_nifgsm")

ni_attack = generate_adv(net_adv_ni, "nifgsm")

train(net_adv_ni, train_loader, train_criterion,val_loader, val_criterion, 
              optimizer, 100, writer, 
                 PATH, True, ni_attack, device="cuda",
                  n_steps_show=1000)

Epoch [0/100]	
Epoch [1/100]	 Training Loss: 2.500364912745288	 lr: 0.001
Epoch [1/100]	 Validation Loss: 1.729025531418716	 lr: 0.001
Epoch [1/100]	
Epoch [2/100]	 Training Loss: 2.138056641649407	 lr: 0.001
Epoch [2/100]	 Validation Loss: 1.6933015086982823	 lr: 0.001
Epoch [2/100]	
Epoch [3/100]	 Training Loss: 2.081118220258552	 lr: 0.001
Epoch [3/100]	 Validation Loss: 1.6526241408118718	 lr: 0.001
Epoch [3/100]	
Epoch [4/100]	 Training Loss: 2.0502456917482266	 lr: 0.001
Epoch [4/100]	 Validation Loss: 1.644403364084944	 lr: 0.001
Epoch [4/100]	
Epoch [5/100]	 Training Loss: 2.027401148815594	 lr: 0.001
Epoch [5/100]	 Validation Loss: 1.6298579505727262	 lr: 0.001
Epoch [5/100]	
Epoch [6/100]	 Training Loss: 2.012241824508628	 lr: 0.001
Epoch [6/100]	 Validation Loss: 1.612912728816648	 lr: 0.001
Epoch [6/100]	
Epoch [7/100]	 Training Loss: 1.999943081375278	 lr: 0.001
Epoch [7/100]	 Validation Loss: 1.6125406539892848	 lr: 0.001
Epoch [7/100]	
Epoch [8/100]	 Training Loss: 1.988

In [31]:
#test clean accuray
test(val_loader, net_adv_ni)

Accuracy of the network on the 10000 test images: 50 %


In [32]:
acc = clean_accuracy(net_adv_ni, adv_images_fgsm_adv, labels)
print('[Model loaded]')
print('Acc: %2.2f %%'%(acc*100))
print("fgsm_sample")

[Model loaded]
Acc: 44.23 %
fgsm_sample


In [33]:
acc = clean_accuracy(net_adv_ni, adv_images_vmifgsm_adv, labels)
print('[Model loaded]')
print('Acc: %2.2f %%'%(acc*100))
print("vmifgsm_sample")

[Model loaded]
Acc: 43.80 %
vmifgsm_sample


In [34]:
acc = clean_accuracy(net_adv_ni, adv_images_nifgsm_adv, labels)
print('[Model loaded]')
print('Acc: %2.2f %%'%(acc*100))
print("nifgsm_sample")

[Model loaded]
Acc: 44.11 %
nifgsm_sample


In [35]:
acc = clean_accuracy(net_adv_ni, adv_images_pgd_adv, labels)
print('[Model loaded]')
print('Acc: %2.2f %%'%(acc*100))
print("pgd_sample")

[Model loaded]
Acc: 44.19 %
pgd_sample


# Adversarial Training by vmifgsm

In [36]:
from torch.utils.tensorboard import SummaryWriter
import copy
net_vmifgsm = copy.deepcopy(net_ori) 

train_criterion = nn.CrossEntropyLoss()
val_criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net_vmifgsm.parameters(), lr=0.001, momentum=0.9)
PATH = './cifar_base_adv_vmifgsm.pth'
device = "cuda"

writer = SummaryWriter("Adv_Base_vmifgsm")
vmi_attack = generate_adv(net_vmifgsm, "vmifgsm")

train(net_vmifgsm, train_loader, train_criterion,val_loader, val_criterion, 
              optimizer, 100, writer, 
                 PATH, True, vmi_attack, device="cuda",
                  n_steps_show=1000)

Epoch [0/100]	
Epoch [1/100]	 Training Loss: 2.6150772364243218	 lr: 0.001
Epoch [1/100]	 Validation Loss: 1.844184164759479	 lr: 0.001
Epoch [1/100]	
Epoch [2/100]	 Training Loss: 2.228119373931299	 lr: 0.001
Epoch [2/100]	 Validation Loss: 1.8138783234584182	 lr: 0.001
Epoch [2/100]	
Epoch [3/100]	 Training Loss: 2.1652783024341553	 lr: 0.001
Epoch [3/100]	 Validation Loss: 1.7679892159715485	 lr: 0.001
Epoch [3/100]	
Epoch [4/100]	 Training Loss: 2.128527822701827	 lr: 0.001
Epoch [4/100]	 Validation Loss: 1.752752812602852	 lr: 0.001
Epoch [4/100]	
Epoch [5/100]	 Training Loss: 2.1025349182241104	 lr: 0.001
Epoch [5/100]	 Validation Loss: 1.7373844083351424	 lr: 0.001
Epoch [5/100]	
Epoch [6/100]	 Training Loss: 2.083202747432777	 lr: 0.001
Epoch [6/100]	 Validation Loss: 1.7255418949489352	 lr: 0.001
Epoch [6/100]	
Epoch [7/100]	 Training Loss: 2.0671914245771323	 lr: 0.001
Epoch [7/100]	 Validation Loss: 1.7078131769276872	 lr: 0.001
Epoch [7/100]	
Epoch [8/100]	 Training Loss: 2

In [42]:
#test clean accuray
test(val_loader, net_vmifgsm)

Accuracy of the network on the 10000 test images: 47 %


In [43]:
acc = clean_accuracy(net_vmifgsm, adv_images_fgsm_adv, labels)
print('[Model loaded]')
print('Acc: %2.2f %%'%(acc*100))
print("fgsm_sample")

[Model loaded]
Acc: 42.77 %
fgsm_sample


In [44]:
acc = clean_accuracy(net_vmifgsm, adv_images_vmifgsm_adv, labels)
print('[Model loaded]')
print('Acc: %2.2f %%'%(acc*100))
print("vmifgsm_sample")

[Model loaded]
Acc: 42.46 %
vmifgsm_sample


In [45]:
acc = clean_accuracy(net_vmifgsm, adv_images_nifgsm_adv, labels)
print('[Model loaded]')
print('Acc: %2.2f %%'%(acc*100))
print("nifgsm_sample")

[Model loaded]
Acc: 42.69 %
nifgsm_sample


In [46]:
acc = clean_accuracy(net_vmifgsm, adv_images_pgd_adv, labels)
print('[Model loaded]')
print('Acc: %2.2f %%'%(acc*100))
print("pgd_sample")

[Model loaded]
Acc: 42.73 %
pgd_sample
